In [1]:
import pandas as pd # Download NLTK stopwords and punkt tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nnarimat/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/nnarimat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_csv("summaries.csv")
df.head()

,Year,Text
0,2004,\n \nScience and Technology in Society forum...
1,2005,\nScience and Technology in Society (STS) for...
2,2006,\nScience and Technology in Society (STS) for...
3,2007,Science and Technology in Society (STS) forum...
4,2008,STS forum 2008 I ...


In [11]:

# Read the CSV file
df = pd.read_csv("summaries.csv")

# Define a function to clean the text
def clean_text(text):
    # Remove newline characters
    text = text.replace('\n', ' ')

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove punctuation and stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]

    # Join the tokens back into a string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Apply the cleaning function to the 'Text' column
df['Cleaned Text'] = df['Text'].apply(clean_text)

# Display the DataFrame with cleaned text
display(df.head(20))

,Year,Text,Cleaned Text
0,2004,\n\nScience and Technology in Society forum\n\...,science technology society forum science techn...
1,2005,\nScience and Technology in Society (STS) foru...,science technology society sts forum lights sh...
2,2006,\nScience and Technology in Society (STS) foru...,science technology society sts forum lights sh...
3,2007,Science and Technology in Society (STS) forum...,science technology society sts forum lights sh...
4,2008,STS forum 2008 I\nScience and Technology...,sts forum science technology society sts forum...
5,2009,Science and Technology in Society\nNon-Profit ...,science technology society organization sts fo...
6,2010,Science and Technology in Society\nNon-Profit ...,science technology society organization sts fo...
7,2011,Science and Technology in Society\nNon-Profit ...,science technology society organization sts fo...
8,2012,II STS forum 2012 ...,ii sts forum sunday october opening plenary se...
9,2013,STS forum – Scien ce and Technology in Societ...,sts forum scien ce technology society forum st...


In [15]:
from collections import Counter

# Define a function to count top 10 words
def count_top_words(text):
    words = text.split()
    word_counts = Counter(words)
    top_10_words = word_counts.most_common(10)
    return dict(top_10_words)

# Apply the function to the 'Cleaned Text' column
df['Top 10 Words'] = df['Cleaned Text'].apply(count_top_words)

# Display the DataFrame with top 10 words for each row

for i in range(20):
    display(df['Top 10 Words'][i])

{'science': 108,
 'technology': 61,
 'session': 49,
 'research': 49,
 'countries': 48,
 'energy': 47,
 'society': 40,
 'said': 40,
 'developing': 39,
 'president': 39}

{'science': 209,
 'technology': 176,
 'society': 84,
 'president': 75,
 'research': 65,
 'forum': 64,
 'us': 64,
 'e': 64,
 'new': 61,
 'jp': 57}

{'e': 232,
 'science': 182,
 'n': 155,
 'r': 145,
 'technology': 124,
 'l': 87,
 'c': 78,
 'h': 77,
 'president': 73,
 'new': 70}

{'science': 221,
 'technology': 180,
 'research': 112,
 'new': 86,
 'us': 86,
 'forum': 83,
 'need': 81,
 'president': 80,
 'innovation': 77,
 'countries': 77}

{'science': 168,
 'technology': 140,
 'research': 123,
 'countries': 105,
 'developing': 93,
 'new': 76,
 'need': 74,
 'president': 73,
 'must': 73,
 'energy': 72}

...

{'science': 290,
 'technology': 282,
 'also': 268,
 'society': 245,
 'research': 221,
 'energy': 172,
 'innovation': 154,
 'data': 154,
 'speakers': 149,
 'japan': 147}

{'science': 374,
 'technology': 276,
 'also': 253,
 'society': 236,
 'pandemic': 214,
 'research': 181,
 'global': 152,
 'data': 140,
 'change': 129,
 'people': 113}

{'science': 313,
 'technology': 211,
 'also': 191,
 'research': 183,
 'society': 135,
 'global': 133,
 'need': 131,
 'new': 125,
 'technologies': 114,
 'must': 108}

{'also': 312,
 'science': 299,
 'technology': 233,
 'research': 219,
 'energy': 160,
 'need': 141,
 'speakers': 135,
 'new': 134,
 'participants': 132,
 'president': 129}

{'also': 324,
 'ai': 280,
 'science': 275,
 'technology': 273,
 'research': 271,
 'participants': 165,
 'speakers': 162,
 'new': 141,
 'data': 139,
 'president': 137}

In [10]:
from collections import Counter

# Define a function to count top 10 words
def count_top_words(text):
    words = text.split()
    word_counts = Counter(words)
    top_20_words = word_counts.most_common(20)
    return dict(top_20_words)

# Apply the function to the 'Cleaned Text' column
df['Top 20 Words'] = df['Cleaned Text'].apply(count_top_words)

# Display the DataFrame with top 20 words for each row

for i in range(20):
    display(df['Year'][i], df['Top 20 Words'][i])

2004

{'science': 108,
 'technology': 61,
 'session': 49,
 'research': 49,
 'countries': 48,
 'energy': 47,
 'society': 40,
 'said': 40,
 'developing': 39,
 'president': 39,
 'new': 36,
 'scientific': 33,
 'public': 31,
 'jp': 30,
 'must': 29,
 'also': 29,
 'government': 28,
 'global': 28,
 'nanotechnology': 28,
 'forum': 26}

2005

{'science': 209,
 'technology': 176,
 'society': 84,
 'president': 75,
 'research': 65,
 'forum': 64,
 'us': 64,
 'e': 64,
 'new': 61,
 'jp': 57,
 'session': 56,
 'human': 56,
 'energy': 55,
 'development': 52,
 'director': 48,
 'university': 47,
 'knowledge': 41,
 'information': 40,
 'countries': 38,
 'global': 37}

2006

...

{'science': 313,
 'technology': 211,
 'also': 191,
 'research': 183,
 'society': 135,
 'global': 133,
 'need': 131,
 'new': 125,
 'technologies': 114,
 'must': 108,
 'innovation': 104,
 'energy': 97,
 'president': 96,
 'world': 93,
 'collaboration': 92,
 'development': 92,
 'quantum': 90,
 'countries': 90,
 'explained': 89,
 'change': 81}

2022

{'also': 312,
 'science': 299,
 'technology': 233,
 'research': 219,
 'energy': 160,
 'need': 141,
 'speakers': 135,
 'new': 134,
 'participants': 132,
 'president': 129,
 'university': 120,
 'chair': 119,
 'must': 113,
 'data': 111,
 'global': 107,
 'technologies': 106,
 'development': 99,
 'education': 96,
 'climate': 95,
 'industry': 93}

2023

{'also': 324,
 'ai': 280,
 'science': 275,
 'technology': 273,
 'research': 271,
 'participants': 165,
 'speakers': 162,
 'new': 141,
 'data': 139,
 'president': 137,
 'must': 134,
 'development': 126,
 'global': 122,
 'need': 118,
 'discussed': 115,
 'university': 114,
 'technologies': 103,
 'challenges': 102,
 'energy': 101,
 'society': 101}